In [18]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import keras
import tensorflow as tf
from keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [22]:
def building_block(filters,block):
    
    if block != 0:
        stride = 1
    else:
        stride = 2
        
    def f(x):
        
        #主通路的网络结构
        y = KL.Conv2D(filters=filters,kernel_size=(1,1),strides=stride)(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)
        
        y = KL.Conv2D(filters=filters,kernel_size=(3,3),padding='same')(y)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)
        
        #主通路的输出
        y = KL.Conv2D(filters=4*filters,kernel_size=(1,1))(y)
        y = KL.BatchNormalization(axis=3)(y)
        
        #判断block的类型
        if block == 0:
            shortcut = KL.Conv2D(filters = 4*filters,kernel_size=(1,1),strides=stride)(x)
            shortcut = KL.BatchNormalization(axis=3)(shortcut)
        else:
            shortcut = x
            
        #主通路与shortcut 相加     
        y = KL.Add()([y,shortcut])
        import random
        y = KL.Activation('relu',name='last'+ str((random.randint(100,300))))(y)
        #y = KL.Activation('relu')(y)
        
        return y
    return f
            

In [15]:
def ResNet_Extractor(inputs):
    
    # reanet 预处理 heading
    x = KL.Conv2D(filters=64,kernel_size=(3,3),padding='same')(inputs)
    x = KL.BatchNormalization(axis=3)(x)
    x = KL.Activation('relu')(x)
    
    filters = 64
    
    block=[2,2,2]
    for i,block_num in enumerate(block):
        for block_id in range(block_num):
            x = building_block(filters=filters,block=block_id)(x)
        filters *=2
        
    return x
    

In [16]:
def RpnNet(inputs,k=9):
    
    #共享层输出
    shareMap = KL.Conv2D(filters=256,kernel_size=(3,3),padding='same',name='sharemap')(inputs)
    shareMap = KL.Activation('linear')(shareMap)  #使用linear
    
    #RPN 分类 前后景
    rpn_classfication = KL.Conv2D(filters=2*k,kernel_size=(1,1))(shareMap)
    # 为了保证与原始分类输入一致，所以需要reshape
    rpn_classfication = KL.Lambda(lambda x:tf.reshape(x,[tf.shape(x)[0],-1,2]))(rpn_classfication)
    rpn_classfication = KL.Activation('linear',name='rpn_classfication')(rpn_classfication)
    
    rpn_probability = KL.Activation('softmax',name='rpn_prob')(rpn_classfication)
    
    # 计算回归修正
    rpn_position = KL.Conv2D(filters=4*k,kernel_size=(1,1))(shareMap)
    rpn_position = KL.Activation('linear')(rpn_position)
    # -l 表示anchor数量不确定
    rpn_BoundingBox = KL.Lambda(lambda x:tf.reshape(x,[tf.shape(x)[0],-1,4]),name='rpn_POS')(rpn_position)
    
    return rpn_classfication,rpn_probability,rpn_BoundingBox


In [23]:
x = KL.Input((64,64,3))
featureMap = ResNet_Extractor(x)
rpn_classfication,rpn_probability,rpn_BoundingBox = RpnNet(featureMap,k=9)
model = Model(inputs=[x],outputs=[rpn_classfication,rpn_probability,rpn_BoundingBox])
plot_model(model=model,to_file='sharemap-64.png',show_shapes=True)

In [ ]:
def RPNClassLoss(rpn_match,rpn_Cal):
    rpn_match = tf.squeeze(rpn_match,axis=-1)
    indices = tf.where(K.not_equal(x=rpn_match,y=0))
    #1=1,0,-1 = 0
    anchor_class = K.cast(K.equal(rpn_match,1),tf.int32)
    
    # 原始样本结果
    anchor_class = tf.gather_nd(params=anchor_class, indices=indices)
    #这个是rpn计算结果
    rpn_cal_class = tf.gather_nd(params=rpn_Cal,indices=indices)
    
    # one hot 
    loss = K.sparse_categorical_crossentropy(target=anchor_class,output=rpn_cal_class,
                                            from_logits=True)
    
    loss = K.switch(condition=tf.size(loss)>0,then_expression=K.mean(loss),else_expression=tf.constant(0.0))
    
    return loss
    